Liveness-, Readiness- und Startup-Tests
=================================

Liveness-Probes (dt: Lebendigkeitstests) helfen, um zu wissen, wann ein Container neu gestartet werden muss. Zum Beispiel könnten Liveness-Probes einen Deadlock abfangen, wenn eine Anwendung ausgeführt wird, aber keine Fortschritte erzielen kann (z.B. abhängige Datenbank startet nicht).

Das nachfolgende Beispiel startet einen Pods welcher nach 15 Sekunden abstürzt, weil die überwachte Datei nicht mehr vorhanden ist:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-exec
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/busybox
    imagePullPolicy: IfNotPresent
    args:
    - /bin/sh
    - -c
    - touch /tmp/healthy; sleep 30; rm -rf /tmp/healthy; sleep 600
    livenessProbe:
      exec:
        command:
        - cat
        - /tmp/healthy
      initialDelaySeconds: 5
      periodSeconds: 5
%EOF%

Das können wir Überprüfen durch mehrmaliges Ausführen von:

In [ ]:
! kubectl describe pod liveness-exec

Ebenfalls, sollte bei der Anzeige des Pods die mehrmaligen Restarts sichtbar sein:

In [ ]:
! kubectl get pods liveness-exec

Aufräumen

In [ ]:
! kubectl delete pod/liveness-exec

***
### Liveness-Tests HTTP Variante

Das gleiche funktioniert auch mit HTTP Ports:


In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f - 
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-http
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/liveness
    imagePullPolicy: IfNotPresent    
    args:
    - /server
    livenessProbe:
      httpGet:
        path: /healthz
        port: 8080
        httpHeaders:
        - name: Custom-Header
          value: Awesome
      initialDelaySeconds: 3
      periodSeconds: 3
%EOF%


Das Programm im Pod ist so geschrieben, dass es die ersten 10 Sekunden eine HTTP Antwort 200 und dann eine HTTP Antwort 500 liefert.

    http.HandleFunc("/healthz", func(w http.ResponseWriter, r *http.Request) {
      duration := time.Now().Sub(started)
      if duration.Seconds() > 10 {
          w.WriteHeader(500)
          w.Write([]byte(fmt.Sprintf("error: %v", duration.Seconds())))
      } else {
          w.WriteHeader(200)
          w.Write([]byte("ok"))
      }
    })
    
    

In [ ]:
! kubectl describe pod liveness-http
! kubectl get pods liveness-http

Aufräumen

In [ ]:
! kubectl delete pods liveness-http

***
Startup-Tests
-------------------

Startup-Probes (dt: Startstests) helfen, um festzustellen, wann eine Containeranwendung gestartet wurde. Wenn eine solche Tests konfiguriert ist, werden die Lebendigkeits- und Bereitschaftsprüfungen deaktiviert, bis sie erfolgreich ist, um sicherzustellen, dass diese Tests den Anwendungsstart nicht stören.

Nehmen wir eine Legacy Anwendung welche einfach eine gewisse Zeit zum Starten braucht. Ein Liveness-Test könnte dazu führen, dass die Anwendung gekillt wird bevor sie überhaupt in einen `ready` Zustand kommen kann.

Das Problem kann mit `startupProbe` Umgangen werden. Dieser stellt sicher, dass `livenessProbe` erst nach `failureThreshold * periodSeconds` ausgeführt wird.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f - 
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-http
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/liveness
    imagePullPolicy: IfNotPresent    
    args:
    - /server
    
    ports:
    - name: liveness-port
      containerPort: 8080
      hostPort: 8080
    
    livenessProbe:
     httpGet:
       path: /healthz
       port: liveness-port
     failureThreshold: 1
     periodSeconds: 10

    startupProbe:
     httpGet:
       path: /healthz
       port: liveness-port
     failureThreshold: 30
     periodSeconds: 10
%EOF%

In [ ]:
! kubectl describe pod liveness-http
! kubectl get pods liveness-http

Aufräumen

In [ ]:
! kubectl delete pods liveness-http

***
Readiness-Tests
---------------

Readiness-Probes (dt: Bereitschaftstests) helfen, um festzustellen, wann ein Container bereit ist, Datenverkehr anzunehmen. Ein Pod gilt als bereit, wenn alle seine Container bereit sind. Wenn ein Pod nicht bereit ist, wird er aus den Service Load Balancern entfernt.

Manchmal können Anwendungen den Datenverkehr vorübergehend nicht bedienen, z.B. weil die Anwendung gerade eine grosse Menge Daten verarbeiten muss. 

Kubernetes soll diese Situtation erkennen und die Anwendung nicht restarten, sondern dafür sorgen, dass sie keine neue Daten empfängt.

Dafür gibt es Readiness-Tests, welche die Anwendung vom Service trennen und so Sicherstellen, dass keine Daten empfangen werden.


In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  labels:
    test: liveness
  name: liveness-exec
spec:
  containers:
  - name: liveness
    image: k8s.gcr.io/busybox
    imagePullPolicy: IfNotPresent
    args:
    - /bin/sh
    - -c
    - touch /tmp/healthy; sleep 30; rm -rf /tmp/healthy; sleep 600
    readinessProbe:
      exec:
        command:
        - cat
        - /tmp/healthy
      initialDelaySeconds: 5
      periodSeconds: 5
%EOF%

In [ ]:
! kubectl describe pod liveness-exec
! kubectl get pods/liveness-exec

Aufräumen

In [ ]:
! kubectl delete pod/liveness-exec